In [1]:
import os

os.chdir('../../Datasets')
os.getcwd()

'/Users/brunogutierrez/workspace/tese/Datasets'

In [2]:
# Standard libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import lines

from nltk.tokenize import sent_tokenize
# import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
raw_path = os.getcwd() + '/Olist/'
order_reviews = pd.read_csv(raw_path + 'olist_order_reviews_dataset.csv')
products = pd.read_csv(raw_path + 'olist_products_dataset.csv')
sellers = pd.read_csv(raw_path + 'olist_sellers_dataset.csv')

In [4]:
# Reading all the files
# raw_path = os.getcwd() + '/Olist/'
# customer = pd.read_csv(raw_path + 'olist_customers_dataset.csv')
# geolocation = pd.read_csv(raw_path + 'olist_geolocation_dataset.csv')
# orders = pd.read_csv(raw_path + 'olist_orders_dataset.csv')
# order_items = pd.read_csv(raw_path + 'olist_order_items_dataset.csv')
# order_payments = pd.read_csv(raw_path + 'olist_order_payments_dataset.csv')
# order_reviews = pd.read_csv(raw_path + 'olist_order_reviews_dataset.csv')
# products = pd.read_csv(raw_path + 'olist_products_dataset.csv')
# sellers = pd.read_csv(raw_path + 'olist_sellers_dataset.csv')

In [5]:
sellers.sample()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
2678,bd1d49a326abfa9b1036772e40445430,17017,bauru,SP


In [9]:
orders = pd.read_csv(raw_path + 'olist_orders_dataset.csv')
orders.sample()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
15246,b41cc5a0872a7fd886b2cb2e300dbe7a,0e7dc1a303c6c24a96103b216d443185,delivered,2018-07-16 10:53:45,2018-07-17 04:31:56,2018-07-23 14:28:00,2018-07-24 14:21:35,2018-07-26 00:00:00


In [8]:
order_items = pd.read_csv(raw_path + 'olist_order_items_dataset.csv')
order_items.sample()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
57536,82e9cdd567caae62a6d0e4e9bbf28901,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-06-22 17:31:06,49.9,20.99


In [10]:
order_reviews.sample()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
75273,6c99a3aef68e5330e0b621b38fd742ce,7b2f3756c235e93b2ad6330c54805401,2,NaN,pois era pra entregar previsão para dia 03/04/...,2018-04-05 00:00:00,2018-04-05 12:59:09


### Número de items por pedido

In [14]:
order_items.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [18]:
# a grande maioria com apenas um pedido
n_itens_per_order = order_items.value_counts('order_id')
n_itens_per_order.describe(percentiles=[.9, .95])

count    98666.000000
mean         1.141731
std          0.538452
min          1.000000
50%          1.000000
90%          1.000000
95%          2.000000
max         21.000000
dtype: float64

### Pedidos que tem produtos de sellers diferentes

1 -  Agrupar order items por order id e contar número de seller_id diferentes

In [30]:
sellers_per_order = order_items.groupby('order_id')['seller_id'].nunique()
multiple_sellers_orders = sellers_per_order[sellers_per_order>1]
print(f'{len(multiple_sellers_orders)} pedidos com mais de um seller.')

1278 pedidos com mais de um seller.


In [35]:
# descartar
filtered_orders_items = order_items[~order_items['order_id'].isin(multiple_sellers_orders.index)]
filtered_orders_items.shape

(109547, 7)

## Alinhar pedidos com reviews

In [54]:
# remover pedidos sem comentário
filtered_order_reviews = order_reviews.dropna(subset='review_comment_message') 
filtered_order_reviews.shape

(41753, 7)

In [55]:
# fazer merge pelo order id com a review
seller_reviews = filtered_orders_items.merge(filtered_order_reviews, on='order_id')
seller_reviews[['order_id', 'seller_id', 'review_score', 'review_comment_message']].sample(5)

,order_id,seller_id,review_score,review_comment_message
40980,e114923f818130cb74183d70a5d22371,01fdefa7697d26ad920e9e0346d4bd1b,5,Produto recebido antes do prazo e conforme des...
45240,f8e3cc453fdc62a2a374893143fcbd09,06579cb253ecd5a3a12a9e6eb6bf8f47,5,Otimo
15133,521e93507df5620aaee015ba441ec756,da8622b14eb17ae2831f4ac5b9dab84a,1,"Está faltando uma cortina, só chego o codgo do..."
23529,81643a310d1fa764a5097fa3545d4a90,817245bcc3badd82bbd222e0366951a6,1,preciso receber minha compra e ela nao passa d...
17894,61b5e6d846c13e8107ec0d264d5abdf1,a5cba26a62b8b4d0145b68b841e62e7f,3,lannister tem nome por isso que gosto de comprar


### Número de comentários por seller

In [56]:
reviews_per_seller = seller_reviews.value_counts('seller_id')
reviews_per_seller.describe()

count    2533.000000
mean       18.369128
std        57.574507
min         1.000000
25%         2.000000
50%         4.000000
75%        13.000000
max       989.000000
dtype: float64

In [57]:
top_sellers = reviews_per_seller[reviews_per_seller>=20]
print(f'{len(top_sellers)} sellers com mais de 20 avaliações')

475 sellers com mais de 20 avaliações


In [58]:
random_seller = top_sellers.sample().index

In [64]:
print(f'seller {random_seller.values[0]}')
seller_reviews.set_index('seller_id').loc[random_seller][['review_comment_message', 'review_score']].values

seller c8b0e2b0a7095e5d8219575d5e7e1181


array([['O produto é bastante confortavel e bonito ,chegou antes do prazo e muito bem embalado.',
        5],
       ['Até estão estou amando meu capacete😊', 5],
       ['Recomendo a todos podem compra sem medo garantido por mim', 5],
       ['Produto entregue antes do prazo! Veio tudo certo!', 5],
       ['Não entregaram o pedido', 1],
       ['Capacete fantástico. Atendimento melhor ainda. Só tenho a agradecer.',
        5],
       ['Minha mercadoria chegou antes do prazo e é exelente ', 1],
       ['O produto chegou corretamente e dentro do prazo', 5],
       ['. Estou chateado com o site. Motivo em que o meu pedido veio diferente do que pedi. E estou com dificuldade para trocar.',
        1],
       ['Mercadoria não foi entregue, lamentável!!!!!', 1],
       ['obrigado,eu recomendo targaryen', 5],
       ['prazo de entrega e produto excelente', 5],
       ['A ENTREGA FOI RAPIDA MUITO BOM', 4],
       ['Produto um pouco fraco, mas atende', 4],
       ['Transmissão top aguenta bastan